# Model regresji liniowej w scikit-learn

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

# Wybór i ocena najlepszego modelu regresji

Podczas budowy modelu, którego celem jest przewidywanie pewnych wartości na podstawie zbioru danych uczących poważnym problemem jest ocena jakości uczenia i zdolności poprawnego przewidywania.

**Częstym błędem osób początkujących w zakresie analizy danych jest przeprowadzanie testów na tym samym zbiorze na którym system był uczony**. Takie rozwiązanie nie jest poprawnym miernikiem jakości nauczonego modelu i prowadzi do wyników które są przeszacowane, czyli nadmiernie optymistyczne.

Zwykle głównym celem budowy modeli predykcyjnych jest późniejsze wykorzystanie przewidywań modelu na danych niedostępnych podczas procesu uczenia więc opracowano szereg metod pozwalających na znacznie bardziej uczciwy pomiar dokładności.

### Podział zbioru na część:
- treningową,
- testową.

Idea oceny modelu lub doboru odpowiednich parametrów modelu sprowadza się wówczas do nauczenia modelu na części uczącej oraz przetestowania go na części testowej, która nie była wykorzystywana w procesie uczenia modelu. Dzięki wydzieleniu dwóch niezależnych podzbiorów, wektory części testowej zawierają informację o faktycznym wyniku jaki powinien zostać osiągnięty, natomiast nauczony (na części uczącej zbioru) model dostarcza wyników przewidywań.

lub

- treningową,
- walidacyjną,
- testową.

Nierzadko, z wydzielenia próby walidacyjnej oraz testowej trzeba zrezygnować i wszystkie dane uznać za elementy próby uczącej. Jak wówczas porównywać różne modele?

Trzeba się odwołać do wielokrotnego wykorzystania elementów próby uczącej, tak przy tym zorganizowanego, by wprowadzane tym sposobem obciążenie otrzymywanych oszacowań było możliwie małe.

## Kroswalidacja - sprawdzanie krzyżowe

1. Próba ucząca zostaje podzielona na $K$ możliwie równych części.

2. Z próby uczącej tworzy się $K$ różnych pseudoprób, powstających przez usuwanie z próby oryginalnej jednej z jej $K$ części. Każda pseudopróba składa się $K-1$ części próby uczącej.

3. Dany model jest budowany $K$-krotnie, za każdym razem na podstawie innej pseudopróby.

4. Otrzymujemy $K$ wersji tego samego modelu.

5. Każda $k$-ta wersja modelu jest oceniana na tej części oryginalnej próby uczącej, która nie weszła do $k$-tej pseudopróby. Tym sposobem, oceny danej wersji modelu dokonujemy na obserwacjach, które nie brały udziałuw jego konstrukcji.

6. Oszacowanie błędu kroswalidacji wyznaczamy jako średnią z błędów każdej wersji modelu.

## Pomiary błędów

### Błąd średniokwadratowy (ang. *Mean squared error*)
$$
\text{MSE} = \frac{1}{n}\sum_{i=1}^n(y_i - \hat{y}_i)^2.
$$

### Mediana błędu bezwzględnego (ang. *Median absolute error*)
$$
\text{MAE}= \text{Med}(|y_i - \hat{y}_i|).
$$

## Zadanie 1
Wczytaj zbiór `Carseats`, a następnie

1. Podziel zbiór na część treningową i testową w stosunku 7:3.

2. Naucz dowolny model na części treningowej:
    - wyznacz błąd dopasowania wykorzystującz MSE i MAE;
    - wyznacz te same błędy wykorzystując metodę kroswalidacji.

3. Sprawdź jakość predykcji. Na podstawie nauczonego modelu na części treningowej, dokonaj predykcji wartość `Sales` dla wartości ze zbioru testowego. Porównaj jakość dopasowania z jakością predykcji.

In [6]:
from sklearn.model_selection import cross_val_score, train_test_split

In [3]:
carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [7]:
# Podział zbioru na część treningową i na część testową
y = carseats.data.Sales
X = carseats.data.iloc[:,1:]
X = pd.get_dummies(X,columns = ['ShelveLoc','Urban','US'],drop_first=True)
print(X.head())
#randomstate bo podział losowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

   CompPrice  Income  Advertising  Population  Price  Age  Education  \
0        138      73           11         276    120   42         17   
1        111      48           16         260     83   65         10   
2        113      35           10         269     80   59         12   
3        117     100            4         466     97   55         14   
4        141      64            3         340    128   38         13   

   ShelveLoc_Good  ShelveLoc_Medium  Urban_Yes  US_Yes  
0           False             False       True    True  
1            True             False       True    True  
2           False              True       True    True  
3           False              True       True    True  
4           False             False       True   False  


In [ ]:
#sprawdzenie
print(X_train.shape[0] / X.shape[0], X_test.shape[0] / X.shape[0])

0.7 0.3


In [ ]:
# Model
lm = LinearRegression()
lm.fit(X=X_train, y=y_train)

LinearRegression()

In [ ]:
import sklearn.metrics as metrics

In [ ]:
#jakośc dopasowania (czyli miary na zb. treningowym)
print("Train R2: {}".format(lm.score(X_train, y_train)))
print("Train MSE: {}".format(metrics.mean_squared_error(y_true=y_train, y_pred=lm.predict(X_train))))
print("Train MAE: {}".format(metrics.median_absolute_error(y_true=y_train, y_pred=lm.predict(X_train))))
print("Train MAR: {}".format(metrics.mean_absolute_error(y_true=y_train, y_pred=lm.predict(X_train))))

Train R2: 0.8877516608074592
Train MSE: 0.9085503897084769
Train MAE: 0.6619545609541619
Train MAR: 0.7652564384688191


In [ ]:
# Za pomocą metody kroswalidacji
lm = LinearRegression()
cv = 10 #liczba podzbiorów biorących udzia w kroswalidacji
print("CV Train R2: {}".format(cross_val_score(lm, X, y, cv=cv).mean()))
print("CV Train MSE: {}".format(-cross_val_score(lm, X, y,
                                           scoring = 'neg_mean_squared_error', cv=cv).mean()))
print("CV Train MAE: {}".format(-cross_val_score(lm, X, y,
                                           scoring = 'neg_median_absolute_error', cv=cv).mean()))
#R^2 maleje, bo jednak jest liczony na części testowej w każdym obrocie pętli kroswalidacyjnej, a MSE i MAE rosną

CV Train R2: 0.8570046698819294
CV Train MSE: 1.0699336881636563
CV Train MAE: 0.7201591261436755


In [ ]:
#jakośc predykcji (na zb. testowym)
lm = LinearRegression()
lm.fit(X=X_train, y=y_train)
print("Test MSE: {}".format(metrics.mean_squared_error(y_true=y_test, y_pred=lm.predict(X_test))))
print("Test MAR: {}".format(metrics.mean_absolute_error(y_true=y_test, y_pred=lm.predict(X_test))))
print("Test MAE: {}".format(metrics.median_absolute_error(y_true=y_test, y_pred=lm.predict(X_test))))
# MSE MAR i MAE rosną

Test MSE: 1.305247392983759
Test MAR: 0.9042605739350009
Test MAE: 0.7750702168988828


# *Zadanie 2
Napisz własną funkcję do kroswalidacji (użyj `from sklearn.model_selection import KFold`).

In [ ]:
from sklearn.model_selection import KFold
#KFold dokonuje podziału na n_splits części
#W każdym obrocie pętli pod test_index są indeksy innej części, a reszta części,
#które pozostają, służy do uczenia

In [ ]:
def cv_fun(model, X, y, cv, score_fun):
    kf = KFold(n_splits=cv)
    scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X=X_train, y=y_train)
        y_pred = model.predict(X_test)
        scores.append(score_fun(y_test, y_pred))
    return np.array(scores)

In [ ]:
cv_fun(lm, X, y, 10, metrics.mean_squared_error).mean()

1.0699336881636563

In [ ]:
cv_fun(lm, X, y, 10, metrics.median_absolute_error).mean()

0.7201591261436755

# Zadanie 3
Dla zbióru `Carseats`,

1. Podziel zbiór na część treningową i testową;

2. Dopasuj model regresji:

a) liniowej `Sales~Price`;

b) liniowej `Sales~Price + Advertising`;

c) liniowej `Sales~.`;

d) wielomianowej stopnia 2 dla zmiennej `Price`;

e) wielomianowej stopnia 3 dla zmiennej `Price`;

3. Wybierz najlepszy model na podstawie miar jakości otrzymanych przy użyciu kroswalidacji 10-krotnej.

4. Dla najlepszego modelu dokonaj predykcji na zbiorze testowy. Wyznacz jakość predykcji.

In [4]:
#zmienne zależne i niezależne i one-hot-encoding
y = carseats.data['Sales']
X = carseats.data.drop('Sales', axis=1)
X = pd.get_dummies(X,columns = ['ShelveLoc','Urban','US'],drop_first=True)

In [ ]:
#1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [ ]:
#2
#Funkcja do tworzenia regresji wielomianowej
#(w szególności stopnia 1 - więczwykłej regresji liniowej)
#a także do liczenia scoreów metodą kroswalidacji

def fit_model(X_train, y_train, degree=1):
    model = make_pipeline(
        PolynomialFeatures(degree=degree,include_bias=False),
        LinearRegression()
    )
    cv_r2 = cross_val_score(model, X_train, y_train, scoring='r2', cv=10).mean()
    cv_mse = -cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=10).mean()
    cv_mae = -cross_val_score(model, X_train, y_train, scoring='neg_median_absolute_error', cv=10).mean()

    return {
        'cv_r2': cv_r2, 'cv_mse': cv_mse, 'cv_mae': cv_mae
    }


In [ ]:
results = pd.DataFrame(index=['cv_r2', 'cv_mse', 'cv_mae'])
results['lm(Sales~Price)'] = fit_model(X_train=np.array(X_train['Price']).reshape(-1, 1), y_train=y_train).values()
results['lm(Sales~Price+Advertising)'] = fit_model(X_train=X_train[['Price', 'Advertising']], y_train=y_train).values()
results['lm(Sales~.)'] = fit_model(X_train=X_train, y_train=y_train).values()
results['poly2(Sales~Price)'] = fit_model(X_train=np.array(X_train['Price']).reshape(-1, 1), y_train=y_train, degree=2).values()
results['poly3(Sales~Price)'] = fit_model(X_train=np.array(X_train['Price']).reshape(-1, 1), y_train=y_train, degree=3).values()
results

,lm(Sales~Price),lm(Sales~Price+Advertising),lm(Sales~.),poly2(Sales~Price),poly3(Sales~Price)
cv_r2,0.134812,0.218384,0.856823,0.133412,0.131569
cv_mse,6.684562,6.060563,0.998997,6.727347,6.703862
cv_mae,1.721873,1.597759,0.719593,1.749296,1.753124


In [ ]:
# 3. Wybierz najlepszy model na podstawie miar jakości otrzymanych przy użyciu kroswalidacji 10-krotnej.
# Najlepsze wyniki na zbiorze treningowym otrzymał model: lm(Sales~.)

In [ ]:
model = LinearRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
#4
#na testowym jakość predykcji
print("Test MSE: {}".format(metrics.mean_squared_error(y_pred=y_pred, y_true=y_test)))
print("Test MAE: {}".format(metrics.mean_absolute_error(y_pred=y_pred, y_true=y_test)))

Test MSE: 1.305247392983756
Test MAE: 0.9042605739350001
